### Clifford QSVT 

In this notebook, I apply the quantum singular value transform to three different clifford circuits.

In [29]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from timeit import timeit

# constructing stabilizer (clifford) circuits
dev1 = qml.device("default.clifford", wires=2)
dev2 = qml.device("default.clifford", wires=2)
dev3 = qml.device("default.clifford", wires=4)

@qml.qnode(dev1)
# clifford 1 has 2 quibits with a depth of 3
def clifford1(return_state=True):
    qml.X(wires=[0])
    qml.CNOT(wires=[0, 1])
    qml.S(wires=[0])
    qml.Hadamard(wires=[1])
    return qml.probs(), ([qml.state()] if return_state else [])

@qml.qnode(dev2)
# clifford 1 has 2 quibits with a depth of 5
def clifford2(return_state=True):
    qml.Hadamard(wires=[0])
    qml.X(wires=[0])
    qml.CY(wires=[0, 1])
    qml.Z(wires=[1])
    qml.S(wires=[1])
    return qml.probs(), ([qml.state()] if return_state else [])

@qml.qnode(dev3)
# clifford 1 has 4 quibits with a depth of 5
def clifford3(return_state=True):
    qml.S(wires=[0])
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=[1])
    qml.CY(wires=[1, 2])
    qml.adjoint(qml.S(wires=[2]))
    qml.CZ(wires=[2, 3])
    return qml.probs(), ([qml.state()] if return_state else [])